# Playing around with PLAsTiCC data
Author: Kara Ponder (SLAC)

This is a playground notebook to understand what needs to be done to get the PLAsTiCC data running. Findings here were pushed to a python script called `use_transformer_plasticc.py` to run fully on the command line. 

Start by redoing some of the normalizations that were originally done in `astrorapid` package. This gives more control over what the data looks like and how it's saved. Still need to run some pieces of `astrorapid` before this starts. Need the astrorapid from K.Ponder here: https://github.com/kponder/astrorapid. 

Much of the code for the data is based on the original `astrorapid` code https://github.com/daniel-muthukrishna/astrorapid.

After creating new PLAsTiCC data, run the transformer model to check preliminary results. Not meant to be used in production.


In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from sklearn.utils import shuffle

import os
import pickle
import copy

The PLAsTiCC training data was generated using `astrorapid`.

In [ ]:
plasticc_training_path = 'plasticc_training_data_dir'
file_extension = '_minmax_yspline_yt0_multiclass_maybefullci()_zNone_bTrue_ig(88, 92, 65, 16, 53, 6).npy'


In [ ]:
single_class=True
normalize=True

In [ ]:
Xerr = np.load(plasticc_training_path + 'Xerr' + file_extension, allow_pickle=True)
X = np.load(plasticc_training_path + 'X' + file_extension, allow_pickle=True)
y = np.load(plasticc_training_path + 'y' + file_extension, allow_pickle=True)

timesX = np.load(plasticc_training_path + 'tinterp' + file_extension, allow_pickle=True)
labels = np.load(plasticc_training_path + 'labels' + file_extension, allow_pickle=True)

#orig_lc = np.load(plasticc_training_path + 'origlc' + file_extension, allow_pickle=True)
with open(plasticc_training_path + 'origlc' + file_extension, 'rb') as f:
    orig_lc = pickle.load(f)


norm_train = np.load(plasticc_training_path + 'normalize_train' + file_extension, allow_pickle=True)
norm_test = np.load(plasticc_training_path + 'normalize_test' + file_extension, allow_pickle=True)


objids_train = np.load(plasticc_training_path + 'objids_train' + file_extension, allow_pickle=True)
objids_test = np.load(plasticc_training_path + 'objids_test' + file_extension, allow_pickle=True)
objids_list = np.load(plasticc_training_path + 'objids' + file_extension, allow_pickle=True)

Augmenting the lightcurves only moves the start data around and does not address the non-representitivty in the data.

This code is based on the original `astrorapid` code https://github.com/daniel-muthukrishna/astrorapid

In [ ]:
def augment_crop_lightcurves(X_local, Xerr_local, y_local, labels_local, timesX_local, orig_lc_local, objids_local):
    X_local = copy.copy(X_local)
    Xerr_local = copy.copy(Xerr_local)
    y_local = copy.copy(y_local)
    labels_local = copy.copy(labels_local)
    timesX_local = copy.copy(timesX_local)
    orig_lc_local = copy.copy(orig_lc_local)
    objids_local = copy.copy(objids_local)

    newX = np.zeros(X_local.shape)
    newXerr = np.zeros(Xerr_local.shape)
    newy = np.zeros(y_local.shape)
    lenX = len(X_local)
    for i in range(lenX):
        if i % 1000 == 0:
            print(f"new {i} of {lenX}")
        #print(np.shape(timesX_local[i]))
        mask = timesX_local[i] >= 0
        nmask = sum(mask)
        newX[i][:nmask] = X_local[i][mask]
        newXerr[i][:nmask] = Xerr_local[i][mask]
        if not single_class:
            newy[i][:nmask] = y_local[i][mask]
        else:
            newy[i] = y_local[i]

    print("Concatenating")
    X_local = np.concatenate((X_local, newX))
    Xerr_local = np.concatenate((Xerr_local, newXerr))
    y_local = np.concatenate((y_local, newy))
    labels_local = np.concatenate((labels_local, labels_local))
    timesX_local = np.concatenate((timesX_local, timesX_local))
    orig_lc_local = orig_lc_local * 2
    objids_local = np.concatenate((objids_local, objids_local))

    return X_local, Xerr_local, y_local, labels_local, timesX_local, orig_lc_local, objids_local

This code is based on the original `astrorapid` code https://github.com/daniel-muthukrishna/astrorapid

In [ ]:
classes = sorted(list(set(labels)))

# Count nobjects per class
for c in classes:
    nobs = len(X[labels == c])
    print(c, nobs)

# Use class numbers 1,2,3... instead of 1, 3, 13 etc.
y_indexes = np.copy(y)
for i, c in enumerate(classes):
    y_indexes[y == c] = i
y = y_indexes

if not single_class:
    y = y + 1

y = to_categorical(y)

# Correct shape for keras is (N_objects, N_timesteps, N_passbands) (where N_timesteps is lookback time)
X = X.swapaxes(2, 1)
Xerr = Xerr.swapaxes(2, 1)

print("Shuffling")
X, Xerr, y, labels, timesX, orig_lc, objids_list = shuffle(X, Xerr, y, labels, timesX, orig_lc, objids_list)
print("Done shuffling")
objids_list = np.array(objids_list)

train_idxes = [i for i, objid in enumerate(objids_list) if objid in objids_train]
test_idxes =  [i for i, objid in enumerate(objids_list) if objid in objids_test]
X_train = X[train_idxes]
Xerr_train = Xerr[train_idxes]
y_train = y[train_idxes]
labels_train = labels[train_idxes]
timesX_train = timesX[train_idxes]
orig_lc_train = [orig_lc[i] for i in train_idxes]
objids_train = objids_list[train_idxes]
X_test = X[test_idxes]
Xerr_test = Xerr[test_idxes]
y_test = y[test_idxes]
labels_test = labels[test_idxes]
timesX_test = timesX[test_idxes]
orig_lc_test = [orig_lc[i] for i in test_idxes]
objids_test = objids_list[test_idxes]


X_train, Xerr_train, y_train, labels_train, timesX_train, orig_lc_train, objids_train = augment_crop_lightcurves(X_train, Xerr_train, y_train, labels_train, timesX_train, orig_lc_train, objids_train)

X_train, Xerr_train, y_train, labels_train, timesX_train, orig_lc_train, objids_train = shuffle(X_train, Xerr_train, y_train, labels_train, timesX_train, orig_lc_train, objids_train)


# Sample weights
counts = np.unique(labels_train, return_counts=True)[-1]
class_weights = max(counts) / counts
class_weights = dict(zip(range(len(counts)), class_weights))
print("Class weights:", class_weights)
l_train_indexes = np.copy(labels_train)
for i, c in enumerate(classes):
    l_train_indexes[l_train_indexes == c] = i
sample_weights = np.zeros(len(l_train_indexes))
for key, val in class_weights.items():
    sample_weights[l_train_indexes == key] = val

# #NORMALISE
if normalize:
    X_train = X_train.copy()
    Xerr_train = Xerr_train.copy()
    X_test = X_test.copy()
    Xerr_test = Xerr_test.copy()

    def do_normalization(d, derr):
        lc_norm = [[min(d[i, :, :].flatten()), max(d[i, :, :].flatten())] for i in range(len(d))]

        for i in range(len(d)):
            wh = np.where((d[i, :, :] > 0.) | (d[i, :, :] < 0.))
            d[i, :, :][wh] =  (d[i, :, :][wh] - lc_norm[i][0]) / (lc_norm[i][1] - lc_norm[i][0])
            derr[i, :, :][wh] =  (derr[i, :, :][wh] - lc_norm[i][0]) / (lc_norm[i][1] - lc_norm[i][0])
        return d, derr, lc_norm

    X_train, Xerr_train, lc_norm_train = do_normalization(X_train, Xerr_train)
    X_test, Xerr_test, lc_norm_test = do_normalization(X_test, Xerr_test)



In [ ]:
## Weight map
X_wgtmap_train = np.zeros(np.shape(Xerr_train))
X_wgtmap_validation = np.zeros(np.shape(Xerr_test))

X_wgtmap_train[np.where(Xerr_train != 0)] = 1.0/Xerr_train[np.where(Xerr_train != 0)]**2 #np.ones(X_train.shape)
X_wgtmap_validation[np.where(Xerr_test != 0)] = 1.0/Xerr_test[np.where(Xerr_test != 0)]**2 #np.ones(X_test.shape)


Save this data so you can use it next time and not rerun these sections.

In [ ]:
training_set_dir = 'your_scratch_location'
file_extension = '_minmax_yspline_yt0_multiclass_maybefullci()_zNone_bTrue_ig(88,92,65,16,53,6).npy'

np.save(os.path.join(training_set_dir,
                    "X_train" + file_extension), X_train)
np.save(os.path.join(training_set_dir,
                    "X_wgtmap_train" + file_extension), X_wgtmap_train)
np.save(os.path.join(training_set_dir,
                    "lc_norm_train" + file_extension), lc_norm_train)
np.save(os.path.join(training_set_dir,
                    "y_train" + file_extension), y_train)
np.save(os.path.join(training_set_dir,
                    "labels_train" + file_extension), labels_train)
np.save(os.path.join(training_set_dir,
                    "timesX_train" + file_extension), timesX_train)
np.save(os.path.join(training_set_dir,
                    "orig_lc_train" + file_extension), orig_lc_train)
np.save(os.path.join(training_set_dir,
                    "objids_train" + file_extension), objids_train)

np.save(os.path.join(training_set_dir,
                    "X_valid" + file_extension), X_test)
np.save(os.path.join(training_set_dir,
                    "X_wgtmap_valid" + file_extension), X_wgtmap_validation)
np.save(os.path.join(training_set_dir,
                    "lc_norm_valid" + file_extension), lc_norm_test)
np.save(os.path.join(training_set_dir,
                    "y_valid" + file_extension), y_test)
np.save(os.path.join(training_set_dir,
                    "labels_valid" + file_extension), labels_test)
np.save(os.path.join(training_set_dir,
                    "timesX_valid" + file_extension), timesX_test)
np.save(os.path.join(training_set_dir,
                    "orig_lc_valid" + file_extension), orig_lc_test)
np.save(os.path.join(training_set_dir,
                    "objids_valid" + file_extension), objids_test)


Before loading in the data, be sure to run the batch size

In [ ]:
batch_size = 64

If you want to use the data set we just generated, run below:

In [ ]:
# Read in dataset
lc_data = X_train

pre_mask_map = np.ma.masked_values(X_train, 0)
mask_map = np.ones(np.shape(X_train))
mask_map[pre_mask_map.mask] = 0.0

pre_mask_map_validation = np.ma.masked_values(X_test, 0)
mask_map_validation = np.ones(np.shape(X_test))
mask_map_validation[pre_mask_map_validation.mask] = 0.0

dataset = tf.data.Dataset.from_tensor_slices((lc_data, mask_map, X_wgtmap_train))


dataset_test = tf.data.Dataset.from_tensor_slices((X_test, mask_map_validation, X_wgtmap_validation))

batch_ds = dataset.batch(batch_size)
batch_ds_valid = dataset_test.batch(batch_size)

If you want to use a previously generated data set, use this cell

In [ ]:
data_dir = 'your/training/data/location'
data_extension = '_nobs50_timestep3_singleclass_noaug_train60_minmax_nospline_v02ci()_zNone_bTrue_ig(88,92,65,16,53,6).npy'


# Read in dataset
_lc_data = np.load(data_dir + 'X_train' + data_extension)
_wgt_map = np.load(data_dir + 'X_wgtmap_train' + data_extension)
_pre_mask_map = np.ma.masked_values(_lc_data, 0)
_mask_map = np.ones(np.shape(_lc_data))
_mask_map[_pre_mask_map.mask] = 0.0

_dataset = tf.data.Dataset.from_tensor_slices((_lc_data, _mask_map, _wgt_map))

_lc_data_valid = np.load(data_dir + 'X_valid' + data_extension)
_wgt_map_valid = np.load(data_dir + 'X_wgtmap_valid' + data_extension)

_pre_mask_map_valid = np.ma.masked_values(_lc_data_valid, 0)
_mask_map_valid = np.ones(np.shape(_lc_data_valid))
_mask_map_valid[_pre_mask_map_valid.mask] = 0.0

_dataset_valid = tf.data.Dataset.from_tensor_slices((_lc_data_valid, _mask_map_valid, _wgt_map_valid))

_batch_ds = _dataset.batch(batch_size)
_batch_ds_valid = _dataset_valid.batch(batch_size)


Or you can use the test data from the repository.

In [ ]:
_lc_data = np.load('lc_data.npy')

_wgt_map = np.load('weightmap.npy')

_mask_map = np.ones(np.shape(_wgt_map))
_mask_map[np.where(_wgt_map == 0)] = 0

_dataset = tf.data.Dataset.from_tensor_slices((_lc_data, _mask_map, _wgt_map))
_batch_ds = _dataset.batch(batch_size)

# Transformer

In [ ]:
import transformer as tran

In [ ]:
# Set parameters
d_model = 128 # input vector must have length d_model
target_vocab_size = 6  # possible results to choose from

lc_length = 100 +1 #50 +1 # light curve length
input_vocab_size = lc_length

## hyperparameters:
num_layers = 6
dropout_rate = 0
dff = 64 # hidden layer size of the feed forward network, needs to be larger than 24, factor of 2^x
num_heads = 8 # d_model % num_heads == 0

kld_alpha = 0.4 
kld_rho = 1e-10

# LC stuff
N_days = 50 + 1
Nf = 6 # number of filters

EPOCHS = 50

Playing with a custom step scheduler. Used in the original Transformer paper (_Attention is all you need_)

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=0):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(tf.multiply(self.d_model, 100000.0)) * tf.math.minimum(arg1, arg2)

learning_rate = CustomSchedule(d_model, warmup_steps=10)
plt.plot(learning_rate(tf.range(100, dtype=tf.float32)))
optimizer = tf.keras.optimizers.Adam(learning_rate)


Define possible loss functions

In [ ]:
class RMSE(tf.keras.losses.Loss):
    def __init__(self, name="rmse"):
        super().__init__(name=name)#

    def call(self, y_true, y_pred):
        mse = tf.math.reduce_mean(tf.square(y_true - y_pred))
        return tf.math.sqrt(mse)

def loss_kld(layer1, layer2, alpha=kld_alpha):
    alpha = tf.constant(alpha, dtype=tf.float32)
    layer1 = layer1[0]
    layer1 = tf.math.abs(layer1)
    layer2 = layer2[0]
    layer2 = tf.math.abs(layer2)

    def loss(y_true, y_pred):
        ones = tf.ones(layer1.shape, dtype=tf.float32)
        rhoc = kld_rho
        rho = rhoc*ones

        def kld(layer):
            kld_1 = tf.math.multiply(rhoc, tf.math.log(tf.math.divide_no_nan(rho, layer)))
            kld_2 = tf.math.multiply((1.0 - rhoc), tf.math.divide_no_nan(tf.math.log(ones-rho), tf.math.log(ones-layer)))
            return tf.reduce_sum(kld_1 + kld_2) #kld_1_without_nans + kld_2_without_nans)

        mse = tf.math.reduce_mean(tf.square(y_true - y_pred))
        rmse = tf.math.sqrt(mse)
        return rmse + tf.multiply(alpha, (kld(layer1) + kld(layer2)))
    return loss

Define the transformer model

In [ ]:

# Define the model
encoder = tran.Encoder(num_layers, d_model, num_heads, dff,
                       lc_length, dropout_rate, embed=True)

decoder = tran.Decoder(num_layers, d_model, num_heads, dff,
                       lc_length, dropout_rate, embed=True)

final_layer = tf.keras.layers.Dense(target_vocab_size)


inp = tf.keras.layers.Input(shape=(None,Nf))
target = tf.keras.layers.Input(shape=(None,Nf))
maskmap = tf.keras.layers.Input(shape=(None,Nf))


x = encoder(inp)
x = decoder(target, x, mask=tran.create_decoder_masks(inp, target))
x = final_layer(x)


mx = tf.keras.layers.Multiply()([x, maskmap])
model = tf.keras.models.Model(inputs=[inp, target, maskmap], outputs=mx)

Implement the loss function and compile

In [ ]:
loss_dict = {'MSE': tf.keras.losses.MeanSquaredError(),
             'MSLE': tf.keras.losses.MeanSquaredLogarithmicError(),
             'Huber': tf.keras.losses.Huber(),
             'MAE': tf.keras.losses.MeanAbsoluteError(),
             'LCE': tf.keras.losses.LogCosh(),
             'RMSE': RMSE(),
             'KLD_RMSE':loss_kld(model.get_layer(name='encoder').get_weights(),
                                 model.get_layer(name='decoder').get_weights()),
             }

loss_object = loss_dict['KLD_RMSE']
train_loss = tf.keras.metrics.Mean(name='train_loss')


def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real,0))
    loss_ = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

# Compile and run the model
model.compile(optimizer=optimizer, loss=loss_function, 
             metrics=[tf.keras.metrics.MeanSquaredError(),
                           tf.keras.metrics.MeanAbsoluteError(),
                           tf.keras.metrics.RootMeanSquaredError(),
                          ])

Create the generator

In [ ]:
num_batches = 0
for (batch, _) in enumerate(_batch_ds):
    num_batches = batch

num_batches_valid = 0
for (batch, _) in enumerate(_batch_ds_valid):
    num_batches_valid = batch

    
# Set up to run the fit
def generator(data_set):
    while True:
        for in_batch, mask_batch, wgt_batch in data_set: 
            yield ( [in_batch , in_batch[:, :-1, :],  mask_batch[:, 1:, :], wgt_batch] , in_batch[:, 1:, :])

Fit the model

In [ ]:
history = model.fit(x = generator(_batch_ds),
                    validation_data = generator(_batch_ds_valid),
                    epochs=50, #EPOCHS,
                    steps_per_epoch=num_batches,
                    validation_steps = num_batches_valid,
                    #verbose=0,
                    )

Plot the loss functions

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

You can save the weights or load in previous weights

In [ ]:
#model.save_weights('your/transformer/weights/transformer.h5')

In [ ]:
#model.load_weights('your/transformer/weights/transformer.h5')


Predict using one of the training data to see if it can at least do that. (last time I checked it could not do it too well)

In [ ]:
loc = 100
decoder_input=[[0.0]*Nf]## or 0
output = tf.expand_dims(decoder_input, 0)

for i in range(N_days-1):
    predictions = model([_lc_data[loc][tf.newaxis, :, :], output, _mask_map[loc][tf.newaxis, 1:i+1, :]]) #.predict
    
    predictions = predictions[: ,-1:, :] ## CHECKKK
    
    output = tf.concat([output, predictions], axis=1)
    tf.squeeze(output, axis=0)

In [ ]:
plt.plot(_lc_data[loc][1:, 0], 's', ls = '-.', color='tab:blue')
plt.plot(_lc_data[loc][1:, 2], 's', ls = '-.', color='tab:orange')
plt.plot(_lc_data[loc][1:, 3], 's', ls = '-.', color='tab:red')

plt.plot(output[0][1:, 2], 'o', lw=2, color='tab:orange')
plt.plot(output[0][1:, 0], 'o', lw=2, color='tab:blue', label='predicted lc') # this might finally be right!
plt.plot(output[0][1:, 1], 'o', lw=2)
plt.plot(output[0][1:, 3], 'o', lw=2, color='tab:red')
plt.plot(output[0][1:, 4], 'o', lw=2)
plt.plot(output[0][1:, 5], 'o', lw=2)

plt.legend()

plt.xlabel('time')
plt.ylabel('brightness')